In [1]:
from pathlib import Path

import pandas as pd
from db import *

/home/jovyan/work/db.py:8: SAWarning: Did not recognize type 'geometry' of column 'geom'
  mapping_hotel = Table(


In [2]:
base_dir = Path(".").resolve()

In [3]:
room_mapping = pd.read_csv(base_dir / "data" / "room_mapping.csv")

In [4]:
room_mapping.head()

,HotelName,Destination,ContractID,OperatorName,ContractSPO,Guarantee,SingleSupplement,SPOName,SPOCode,contractType,...,MinRoomPrice,MaxOccupancy,MinOccupancy,MaxAdl1,MinAdl,MaxChl,MinChl,AdultOnly1,InfantAllowed1,MealPlan
0,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,294990,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,2,2,2,2,2,0,0,True,False,BB
1,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,301334,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,2,2,2,2,2,0,0,True,False,BB
2,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,294990,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,2,3,2,3,2,1,0,True,False,BB
3,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,301334,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,2,3,2,3,2,1,0,True,False,BB
4,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,294990,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,2,2,2,2,2,0,0,True,False,BB


In [5]:
hotel_mapping = pd.read_sql(sql=select(mapping_hotel), con=engine.connect())

In [6]:
hotel_mapping.head()

,id,hotel_name,is_charter,is_active,hotel_id
0,90004,JUWEIRA HYDRO CHARTER 17-18,True,False,10001048
1,90012,CLUB ROTANA FISCHER,False,False,10001006
2,90013,FANAR HYDRO CHARTER 17-18,True,False,10001020
3,90016,FANAR KAPPA CLUB,False,False,10001020
4,90021,ROUNDTRIP CUSTOM PREF PARTIES,False,True,10001080


In [25]:
id_mapping = dict(zip(hotel_mapping["hotel_name"], hotel_mapping["hotel_id"]))

In [8]:
room_mapping["hotel_id"] = room_mapping["HotelName"].map(id_mapping)

In [9]:
room_mapping.head()

,HotelName,Destination,ContractID,OperatorName,ContractSPO,Guarantee,SingleSupplement,SPOName,SPOCode,contractType,...,MaxOccupancy,MinOccupancy,MaxAdl1,MinAdl,MaxChl,MinChl,AdultOnly1,InfantAllowed1,MealPlan,hotel_id
0,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,294990,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,2,2,2,2,0,0,True,False,BB,10000072.0
1,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,301334,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,2,2,2,2,0,0,True,False,BB,10000072.0
2,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,294990,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,3,2,3,2,1,0,True,False,BB,10000072.0
3,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,301334,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,3,2,3,2,1,0,True,False,BB,10000072.0
4,25HOURS HOTEL DUBAI ONE CENTRAL,DUBAI,294990,MEETING POINT EMIRATES,Main Contract,False,False,NaN,NaN,P,...,2,2,2,2,0,0,True,False,BB,10000072.0


In [10]:
room_mapping.drop_duplicates(
    subset=["hotel_id", "MapRoomCode"], keep="last", inplace=True
)

In [11]:
import re

replacement_mapping = {
    r"Single Use": " ",
    r"Single": " ",
    r"Singe": " ",
    r"\( \)": " ",
    r"Twin Rm/": "Twin Room",
    r"Bedrooms": "Bedroom",
    r"01": "1",
    r"02": "2",
    r"03": "3",
}


def replace_strings(s):
    for pattern, replacement in replacement_mapping.items():
        s = re.sub(pattern, replacement, s, flags=re.IGNORECASE)
    return s.strip()


room_mapping["HotelRoomDescription"] = room_mapping["HotelRoomDescription"].apply(
    replace_strings
)
room_mapping["HotelRoomDescription"] = (
    room_mapping["HotelRoomDescription"].str.split().str.join(" ").str.title()
)

In [12]:
room_mapping_unique = (
    room_mapping.groupby(["HotelRoomDescription", "hotel_id"]).size().reset_index()
)
room_mapping_unique.sort_values(["hotel_id", "HotelRoomDescription"], inplace=True)
room_mapping_unique.to_csv("room_mapping_unique.csv", index=False)

In [13]:
room_mapping.to_csv("room_mapping.csv", index=False)

In [14]:
room_mapping_unique.to_csv("room_mapping_unique.csv", index=False)

In [15]:
import json

output = []

for _, row in room_mapping_unique.iterrows():
    hotel_room_dict = {
        "model": "accommodation.hotelroom",
        "fields": {
            "hotel": row["hotel_id"],
            "name": row["HotelRoomDescription"],
        },
    }
    output.append(hotel_room_dict)

output_json = json.dumps(output, indent=2, default=str)

with open(base_dir / "fixtures" / "hotelroom.json", "w") as f:
    f.write(output_json)

In [16]:
accommodation_hotel_room = pd.read_sql(
    sql=select(accommodation_hotel_room), con=engine.connect()
)

In [17]:
accommodation_hotel_room.head()

,id,name,hotel_id
0,1,Artist Village Suite,10000072
1,2,Bedouin Room,10000072
2,3,Farmstay Suite,10000072
3,4,Farmstay Suite King,10000072
4,5,Glamping King Room,10000072


In [18]:
id_mapping = dict(
    zip(
        accommodation_hotel_room.apply(lambda x: (x["name"], x["hotel_id"]), axis=1),
        accommodation_hotel_room["id"],
    )
)

In [19]:
room_mapping["room_id"] = room_mapping.apply(
    lambda x: id_mapping.get(
        (
            x["HotelRoomDescription"],
            x["hotel_id"],
        )
    ),
    axis=1,
)

In [20]:
room_mapping.dropna(subset=["hotel_id", "room_id"], inplace=True)
room_mapping["room_type"] = room_mapping["RoomDescription"].str.cat(
    room_mapping["RoomView"].fillna(""), sep=" "
)

In [21]:
output = []

for _, row in room_mapping.iterrows():
    hotel_room_mapping_dict = {
        "model": "mapping.hotelroommapping",
        "fields": {
            "hotel_room": row["room_id"],
            "room_type": row["room_type"],
            "room_code": row["MapRoomCode"],
        },
    }
    output.append(hotel_room_mapping_dict)

output_json = json.dumps(output, indent=2, default=str)

with open("fixtures/hotelroommapping.json", "w") as f:
    f.write(output_json)

In [22]:
room_mapping.to_csv("room_mapping.csv", index=False)